In [54]:
import numpy as np
import pandas as pd
import os
from pyplink import PyPlink
from basic_tools import *

In [2]:
#plink=PyPlink("data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged")

# Individuals common in data

In [3]:
pheno_ind=pd.read_csv(pheno_all_file_path,sep='\t')["ID"]

In [4]:
_1000G_ind=pd.read_csv('data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.fam',sep=' ',header=None)[0]

In [7]:
bmarker_ind=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.fam',sep=' ',header=None)[0]

In [8]:
individual_common=pd.DataFrame(pd.Index(pheno_ind).intersection(pd.Index(_1000G_ind)).intersection(pd.Index(bmarker_ind))).sort_values(0)[[0,0]]

In [35]:
individual_common.to_csv(individual_common_path,sep='\t',index=None,header=None)

In [11]:
#individual_common

# Imputation(CookHLA) included in HATK

HATK returns `*.ped` and `*.alleles`

# NomenCleaner

# BmarkerGenerator

# Beagle phasing

# AA call (deprecated)
run_subprocess(' '.join(['Rscript AllCC_Get_Phased_AA_Calls.R',
                bmarker_phased_path,
                bmarker_plink_path+'.fam',
                bmarker_aa_path                        
                        ]),dry=True)

# KCHIP + 1000G

## 1000G

In [45]:
command='plink --vcf data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.recode.vcf.gz \
--keep-allele-order \
--make-bed \
--out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC'.format(individual_common_path)
command

'plink --vcf data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.recode.vcf.gz --keep-allele-order --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC'

In [52]:
frq=pd.read_csv('data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.frq',sep=r'\s*')

/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [55]:
#frq.sort_values('MAF')

In [46]:
command='plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC \
--keep {} \
--keep-allele-order \
--make-bed \
--out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter'.format(individual_common_path)
command

'plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC --keep data/individual.tsv --keep-allele-order --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter'

1000G
MAF 0.01
Imputation quality score 0.8

hwe 1e-10

In [12]:
_1000G_bim=pd.read_csv('data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter'+'.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])
_1000G_bim.shape

(63544, 6)

In [13]:
bmarker_bim=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele'+'.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])
bmarker_bim.shape

(12101, 6)

In [14]:
_1000G_bmarker=_1000G_bim.rename(columns=lambda x: str(x)+'_1000G').merge(right=bmarker_bim.rename(columns=lambda x: str(x)+'_bmarker'),how='inner',left_on='BP_1000G',right_on='BP_bmarker')

In [26]:
_1000G_bmarker['SNP_1000G'].to_csv('data/genotype/0_1000G/1000G_bmarker_overlap.tsv',index=None,header=None)

/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [27]:
run_subprocess('plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter --exclude data/genotype/0_1000G/1000G_bmarker_overlap.tsv --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.nooverlap',dry=True)

------RUN-----
plink --bfile data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter --exclude data/genotype/0_1000G/1000G_bmarker_overlap.tsv --make-bed --out data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.nooverlap


## KCHIP

In [40]:
command="plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele \
--keep {} \
--freq \
--keep-allele-order \
--out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele".format(individual_common_path)
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele --keep data/individual.tsv --freq --keep-allele-order --out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele'

In [45]:
command="plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele \
--keep {} \
--maf 0.005 \
--make-bed \
--keep-allele-order \
--freq \
--out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC".format(individual_common_path)
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele --keep data/individual.tsv --maf 0.005 --make-bed --keep-allele-order --freq --out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC'

In [16]:
bmarker_qc_bim=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC'+'.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])

In [22]:
bmarker_qc_bim[(bmarker_qc_bim.SNP.str.slice(stop=3)=='AA_') | (bmarker_qc_bim.SNP.str.slice(stop=3)=='HLA')]['SNP'].to_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.HLA_aa_list.tsv',index=None,header=None)

bmarker_qc_bim[(bmarker_qc_bim.SNP.str.slice(stop=3)=='AA_')]['SNP'].to_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.aa_list.tsv',index=None,header=None)


/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
/home/ch6845/tools/miniconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
command="plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC \
--exclude data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.HLA_aa_list.tsv \
--make-bed \
--keep-allele-order \
--out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noHLAaa"
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC --exclude data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.HLA_aa_list.tsv --make-bed --keep-allele-order --out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noHLAaa'

In [24]:
command="plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC \
--exclude data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.aa_list.tsv \
--make-bed \
--keep-allele-order \
--out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa"
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC --exclude data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.aa_list.tsv --make-bed --keep-allele-order --out data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa'

In [25]:
before_maf=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])

In [26]:
after_maf=pd.read_csv('data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.bim',sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])

In [27]:
(after_maf.SNP=='rs2523942').sum()

1

In [34]:
removed=pd.Index(before_maf.SNP).difference(after_maf.SNP)
removed=pd.Series(removed)

In [36]:
#removed_aaformat=['_'.join(i.split('_')[:3]) if i[:3]=='AA_' else i for i in removed]
#removed_aaformat_nodup=pd.Series(removed_aaformat)[~pd.Series(removed_aaformat).duplicated(keep=False)].values

In [37]:
removed

0              AA_A_-22_30018317_exon1
1               AA_A_19_30018567_exon2
2              AA_A_236_30020038_exon4
3              AA_A_245_30020065_exon4
4              AA_A_268_30020134_exon4
                    ...               
557    SNPS_DRB1_4380_32661118_intron1
558    SNPS_DRB1_6220_32659278_intron2
559      SNPS_DRB1_7920_32657578_exon3
560    SNPS_DRB1_8565_32656933_intron3
561    SNPS_DRB1_9302_32656196_intron4
Name: SNP, Length: 562, dtype: object

In [38]:
len(removed)

562

## iid matching

In [30]:
individual_common=pd.read_csv(individual_common_path,sep='\t',header=None)[0]

In [31]:
f_in=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased",'r')
iid_df=pd.DataFrame(f_in.readline().strip().split(' ')[2:],columns=['iid']).reset_index()
f_in.close()

In [32]:
iid_df.shape[0]/2

125872.0

In [39]:
iid_df_merged=pd.DataFrame(individual_common).merge(right=iid_df,right_on='iid',left_on=0,how='inner')

In [43]:
#removed

In [41]:
assert np.all(iid_df_merged.iid.unique()==individual_common)

In [52]:
'AA_A_-22_30018317_exon1' in removed.values

True

In [53]:
f_in=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased",'r')
f_out=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.QC",'w')

ind_count=None
line_cnt=0
total_removed=0

while True:
    line_cnt+=1
    line=f_in.readline().strip()
    line_split=line.split(' ')
    if not line:
        break
    if ind_count is None:
        ind_count=int(len(line_split[2:])/2)
    else:
        assert ind_count==int(len(line_split[2:])/2)
    
    line_type=line_split[0]
    line_id=line_split[1]
    line_data=line_split[2:]
    line_data=np.array(line_data)[iid_df_merged['index'].tolist()]
    
    if line_id in removed.values:
        total_removed+=1
        print(line_id,"remoed")
    else:
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
print(total_removed,"removed")        
f_in.close()
f_out.close()

HLA_A*01:03 remoed
HLA_A*02:02 remoed
HLA_A*02:05 remoed
HLA_A*02:09 remoed
HLA_A*02:10 remoed
HLA_A*02:11 remoed
HLA_A*03:02 remoed
HLA_A*11:02 remoed
HLA_A*11:03 remoed
HLA_A*11:12 remoed
HLA_A*11:77 remoed
HLA_A*23:01 remoed
HLA_A*24:03 remoed
HLA_A*24:04 remoed
HLA_A*24:07 remoed
HLA_A*24:08 remoed
HLA_A*24:10 remoed
HLA_A*24:20 remoed
HLA_A*25:01 remoed
HLA_A*29:02 remoed
HLA_A*31:02 remoed
HLA_A*34:01 remoed
HLA_A*68:01 remoed
HLA_A*68:71 remoed
HLA_A*69:01 remoed
SNPS_A_7_30018316_exon1 remoed
AA_A_-22_30018317_exon1 remoed
AA_A_3_30018519_exon2 remoed
SNPS_A_211_30018520_exon2 remoed
SNPS_A_257_30018566_exon2 remoed
AA_A_19_30018567_exon2 remoed
AA_A_43_30018639_exon2 remoed
SNPS_A_330_30018639_exon2 remoed
SNPS_A_773_30019082_exon3 remoed
SNPS_A_1567_30019876_intron3 remoed
AA_A_236_30020038_exon4 remoed
SNPS_A_1729_30020038_exon4 remoed
AA_A_245_30020065_exon4 remoed
SNPS_A_1756_30020065_exon4 remoed
SNPS_A_1824_30020133_exon4 remoed
AA_A_268_30020134_exon4 remoed
AA_A_283_30

# Merge

_1000G_nooverlap=PyPlink("data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.nooverlap")

_1000G_nooverlap_bim=_1000G_nooverlap.get_bim()
_1000G_nooverlap_fam=_1000G_nooverlap.get_fam()

_1000G_nooverlap_bim

np.all(iid_df_merged['iid']==np.repeat(_1000G_nooverlap_fam['iid'].values,2))

f=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.QC_",'a')

for idx_bim,(SNP,row) in enumerate(_1000G_nooverlap_bim.iterrows()):
    if len(np.unique(_1000G_nooverlap.get_geno_marker(SNP)))!=3:
        print(SNP,'may induce some error')
    line_type='N'
    line_id=SNP+'_'+SNP[-1]
    line_data=np.repeat(_1000G_nooverlap.get_geno_marker(SNP),2).astype('str')
    
    f.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')

f.close()

f_in=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased",'r')
f_out=open("data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.QC",'w')

ind_count=None
line_cnt=0
total_removed=0

while True:
    line_cnt+=1
    line=f_in.readline().strip()
    line_split=line.split(' ')
    if not line:
        break
    if ind_count is None:
        ind_count=int(len(line_split[2:])/2)
    else:
        assert ind_count==int(len(line_split[2:])/2)
    
    line_type=line_split[0]
    line_id=line_split[1]
    line_data=line_split[2:]
    line_data=np.array(line_data)[iid_df_merged['index'].tolist()]
    
    if line_id in removed.values:
        total_removed+=1
        print(line_id,"remoed")
    else:
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
print(total_removed,"removed")        
f_in.close()
f_out.close()

for idx_bim,(SNP,row) in enumerate(_1000G_nooverlap_bim.iterrows()):
    if len(np.unique(_1000G_nooverlap.get_geno_marker(SNP)))!=3:
        print(SNP)

_1000G_nooverlap_bim.index.str.slice(start=-1)

SNP="6:28000361_T/C"
#SNP="6:28000610_C/A"

_1000G_nooverlap_bim.index

np.any(_1000G_nooverlap_bim.index.str.slice(start=-1)==_1000G_nooverlap_bim['a1'])

_1000G_nooverlap_bim

dosage=_1000G_nooverlap.get_geno_marker(SNP)
dosage
(2*(dosage==2).sum()+(dosage==1).sum())/(len(dosage)*2)
dosage

_1000G_nooverlap.get_acgt_geno_marker(SNP)

            
aa_marker_name_list_aaonly=pd.Series(aa_marker_name_list)[pd.Series(aa_marker_name_list).str.slice(stop=3)=='AA_'].values            

_1000G_nooverlap_fam['iid']

pd.Series(_1000G_nooverlap.get_geno_marker('6:28000361_T/C')).value_counts()

(2*1089+21692)/(2*125673)

_1000G_nooverlap.get_geno_marker('6:28000361_T/C').shape

pd.Series(_1000G_nooverlap.get_geno_marker('6:28000610_C/A')).value_counts()

_1000G_nooverlap_fam

In [28]:
!echo 'data/genotype/0_1000G/KCHIP_HLA_6_28Mb_35Mb_20200113_QC.newfilter.nooverlap' > data/genotype/4_merge/merge_list.tsv

In [46]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC \
--merge-list data/genotype/4_merge/merge_list.tsv \
--make-bed \
--keep-allele-order \
--out data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC --merge-list data/genotype/4_merge/merge_list.tsv --make-bed --keep-allele-order --out data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged'

In [138]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC \
--make-bed \
--keep-allele-order \
--out data/genotype/4_merge/KCHIP_HLA_AA_SNP'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC --make-bed --keep-allele-order --out data/genotype/4_merge/KCHIP_HLA_AA_SNP'

In [47]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa \
--merge-list data/genotype/4_merge/merge_list.tsv \
--make-bed \
--keep-allele-order \
--out data/genotype/4_merge/KCHIP_HLA_SNP_1000G_merged'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa --merge-list data/genotype/4_merge/merge_list.tsv --make-bed --keep-allele-order --out data/genotype/4_merge/KCHIP_HLA_SNP_1000G_merged'

In [139]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa \
--make-bed \
--keep-allele-order \
--out data/genotype/4_merge/KCHIP_HLA_SNP'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noaa --make-bed --keep-allele-order --out data/genotype/4_merge/KCHIP_HLA_SNP'

In [48]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noHLAaa \
--merge-list data/genotype/4_merge/merge_list.tsv \
--make-bed \
--keep-allele-order \
--out data/genotype/4_merge/KCHIP_SNP_1000G_merged'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noHLAaa --merge-list data/genotype/4_merge/merge_list.tsv --make-bed --keep-allele-order --out data/genotype/4_merge/KCHIP_SNP_1000G_merged'

In [140]:
command='plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noHLAaa \
--make-bed \
--keep-allele-order \
--out data/genotype/4_merge/KCHIP_SNP'
command

'plink --bfile data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.QC.noHLAaa --make-bed --keep-allele-order --out data/genotype/4_merge/KCHIP_SNP'

In [121]:
! cp data/genotype/3_KCHIP_bmarkerphased/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.QC \
data/genotype/4_merge/KCHIP_HLA.hg18.intersection_HAN.LABELED.NoSameAllele.bMarkers.2field.saveRareAllele.beagle5.1.AGM.bgl.phased.QC

In [65]:
#bim_read=pd.read_csv("data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged.bim",header=None,sep='\t',names=['CHR','SNP','CM','BP','A1','A2'])

In [76]:
#bim_read[(bim_read['SNP'].str.slice(stop=3)=='HLA')|(bim_read['SNP'].str.slice(stop=3)=='AA_')]['SNP'].to_csv("HLA_aa_list.tsv",sep='\t',index=None,header=None)

In [59]:
command='gcta64 --bfile {} --make-grm --out {} --thread-num 40'.format(final_plink_aa_path,final_plink_aa_path+'.grm')
command

'gcta64 --bfile data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged --make-grm --out data/genotype/4_merge/KCHIP_HLA_AA_SNP_1000G_merged.grm --thread-num 40'

In [78]:
command='gcta64 --bfile {} --make-grm --out {} --thread-num 40'.format('data/genotype/4_merge/KCHIP_SNP_1000G_merged','data/genotype/4_merge/KCHIP_SNP_1000G_merged'+'.grm')
command

'gcta64 --bfile data/genotype/4_merge/KCHIP_SNP_1000G_merged --make-grm --out data/genotype/4_merge/KCHIP_SNP_1000G_merged.grm --thread-num 40'

In [137]:
f_in=open("data/genotype/4_merge/KCHIP_HLA_AA_SNP.bgl.phased",'r')
f_out=open("data/genotype/4_merge/KCHIP_HLA_AA.bgl.phased",'w')


IID_list=[]
sex_list=[]
marker_name_list=[]
marker_data_list=[]

line_cnt=0
while True:
    if line_cnt%100==0:
        print(line_cnt,end=' ')
    line_cnt+=1
    line=f_in.readline().strip()
    if not line:
        break
        
    line_split=line.split(' ')
    line_type,line_id,line_data=line_split[0],line_split[1],np.array(line_split[2:])
    
    
    if len(IID_list)>0:
        assert len(line_data)==2*len(IID_list)
    
    if line_type=='P':
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
    elif line_type=='fID':
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
    elif line_type=='mID':
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
    elif line_type=='I':        
        IID_list1=np.array([line_data[i] for i in range(0,len(line_data),2)])
        IID_list2=np.array([line_data[i+1] for i in range(0,len(line_data),2)])
        if np.all(IID_list1==IID_list2):
            IID_list=IID_list1
        else:
            raise
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')
    elif line_type=='C':
        sex_list1=np.array([line_data[i] for i in range(0,len(line_data),2)])
        sex_list2=np.array([line_data[i+1] for i in range(0,len(line_data),2)])
        if np.all(sex_list1==sex_list2):
            sex_list=sex_list1.astype(int)
        else:
            raise
        f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')    
    elif line_type=='M':
        marker_name_list.append(line_id)
        marker_data_list.append(line_data)
        if line_id[:3]=='HLA' or line_id[:3]=='AA_':
            print(line_id)
            f_out.write(' '.join([line_type,line_id]+line_data.tolist())+'\n')    
    else:
        print(line_type)
        raise     

f_in.close()
f_out.close()

0 100 200 300 400 HLA_A*01:01
HLA_A*02:01
HLA_A*02:03
HLA_A*02:06
HLA_A*02:07
HLA_A*03:01
HLA_A*11:01
HLA_A*24:02
HLA_A*26:01
HLA_A*26:02
HLA_A*26:03
HLA_A*29:01
HLA_A*30:01
HLA_A*30:04
HLA_A*31:01
HLA_A*32:01
HLA_A*33:03
AA_A_-15_30018338_exon1
AA_A_-11_30018350_exon1
AA_A_9_30018537_exon2_F
AA_A_9_30018537_exon2_S
AA_A_9_30018537_exon2_T
AA_A_9_30018537_exon2_Y
AA_A_17_30018561_exon2
AA_A_44_30018642_exon2
AA_A_56_30018678_exon2
AA_A_62_30018696_exon2_E
AA_A_62_30018696_exon2_G
AA_A_62_30018696_exon2_L
AA_A_62_30018696_exon2_Q
AA_A_62_30018696_exon2_R
500 AA_A_63_30018699_exon2_E
AA_A_63_30018699_exon2_N
AA_A_63_30018699_exon2_Q
AA_A_65_30018705_exon2
AA_A_66_30018708_exon2
AA_A_67_30018711_exon2
AA_A_70_30018720_exon2
AA_A_73_30018729_exon2
AA_A_74_30018732_exon2
AA_A_76_30018738_exon2_A
AA_A_76_30018738_exon2_E
AA_A_76_30018738_exon2_V
AA_A_77_30018741_exon2_D
AA_A_77_30018741_exon2_N
AA_A_77_30018741_exon2_S
AA_A_79_30018747_exon2
AA_A_80_30018750_exon2
AA_A_81_30018753_exon2
AA_A